In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "3"
os.environ["CUDA_VISIBLE_DEVICES"]
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
#!pip3 install pytorch
!pip3 install multiprocess
import torch
torch.cuda.empty_cache()

#!pip3 install sentence-transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator, CEBinaryClassificationEvaluator
from torch.utils.data import DataLoader
import math
import plotly
import plotly.graph_objs as go
import plotly.io as pio
plotly.offline.init_notebook_mode(connected=True)
import multiprocess as multiprocessing
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import logging
!pip3 install transformers
import json
import pickle
import numpy as np
from tqdm.notebook import trange, tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from writeprints import get_writeprints_transformer, prepare_entry
from utills import batch
from pytorch_models import NeuralNet
import torch
import torch.nn as nn
from torch.utils import data
import copy
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from transformers import BertTokenizerFast
import datetime
import time
from pan20_verif_evaluator import evaluate_all
import nltk
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt to
[nltk_data]     /storagenfs/g.pisciotta1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Constants
DATA_DIR = 'data/small/'
TEMP_DATA_DIR = 'temp_data/pan20_computed/'

Load ground truth
====

In [4]:
# Load gound truth
ground_truth = {}
partition = {}

#tot = 52601/3
limited = False
tot = 20000
n_of_pos_we_want = tot/2
n_of_neg_we_want = tot/2

# quindi in totale 1_000
positive_samples = 0
negative_samples = 0
total = 0

# Qui apriamo il dataset e estraiamo un totale di 10k esempi del task
with open(DATA_DIR + '/pan20-authorship-verification-training-small-truth.jsonl', 'r') as f:
    
    for counter, l in tqdm(enumerate(f)):
        total += 1
        
        d = json.loads(l)

        if d['same'] and positive_samples < n_of_pos_we_want : # se la label è true e dobbiamo ancora aggiungerne
            ground_truth[d['id']] = d['same']
            positive_samples += 1
        elif not d['same'] and negative_samples < n_of_neg_we_want:
            ground_truth[d['id']] = d['same']
            negative_samples += 1
            
        # Una volta che abbiamo raggiunto il numero di dati che vogliamo, è fatta, usciamo
        if limited and (positive_samples == n_of_pos_we_want and negative_samples == n_of_neg_we_want):
            break
            
print(total, positive_samples, negative_samples)


52601 10000 10000


Partially extract features
===

In [5]:
samples = multiprocessing.Manager().Queue()

from sklearn.model_selection import train_test_split                 
def process_pair(l):
    d = json.loads(l)
    if d['id'] in ground_truth:
        e1 = prepare_entry(d['pair'][0])
        e2 = prepare_entry(d['pair'][1])
        samples.put({'id': d['id'], 'doc1': e1, 'doc2': e2})
        return {'id': d['id'], 'doc1':e1, 'doc2':e2}    


start_time = datetime.datetime.now()
print("Started handling of each pair")
with open(DATA_DIR + 'pan20-authorship-verification-training-small.jsonl', 'r') as f:
    with multiprocessing.Pool() as pool:
        samples = pool.map(process_pair, (l for l in f))
print("Ended handling of each pair")        

end_time = datetime.datetime.now()
print("Time spent: ", (end_time-start_time).total_seconds())



Started handling of each pair
Ended handling of each pair
Time spent:  212.726308


Split dataset
=====

In [6]:
labels = []

# Il processo in parallelo restituisce comunque eventualmente dei None se 
# il sample non appartiene alla lista di quelli che abbiamo selezionato,
# quindi nel caso rimuoviamo i None così
samples = [i for i in samples if i is not None]

for s in samples:
    # Qui non dovrebbe arrivare
    if s is None:
        print("None")
        break
        
    # Questo dovrebbe essere il caso normale
    else:
        s['label'] = int(ground_truth[s['id']])

import pandas as pd
df = pd.DataFrame(samples)
df = df.set_index('id')

X_design, X_test, y_design, y_test = train_test_split(df, df['label'], stratify=df['label'], random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_design, y_design, stratify=y_design, random_state=42)

# Per salvare
X_test.to_pickle(TEMP_DATA_DIR+"x_test.pkl")
X_valid.to_pickle(TEMP_DATA_DIR+"x_valid.pkl")
X_train.to_pickle(TEMP_DATA_DIR+"x_train.pkl")
df.to_pickle(TEMP_DATA_DIR+"df.pkl")


In [7]:
df

,doc1,doc2,label
id,,,
6cced668-6e51-5212-873c-717f2bc91ce6,"{'preprocessed': 'I shift a bit, warily lettin...","{'preprocessed': '""All will become one with Ru...",1
3c6c188a-db28-59aa-8c09-3d0f799ff579,"{'preprocessed': 'I shift a bit, warily lettin...","{'preprocessed': 'Suddenly, a piece of ice fal...",1
b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,{'preprocessed': 'A single tear escaped me as ...,"{'preprocessed': 'got the Yang yoyo."" Kimiko p...",1
e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,"{'preprocessed': '""Ja."" Ludwig kept his gaze u...",{'preprocessed': 'SilverGray lll...YellowRagge...,1
4fe541af-912e-5a86-81a5-94c6d3891509,"{'preprocessed': 'And he did. Slowly, hesitant...","{'preprocessed': '""Let""s go,"" Raimondo said an...",1
...,...,...,...
d244814f-fbf1-5db4-a40c-b973b08ac9f8,{'preprocessed': 'He had a nice voice. Althoug...,{'preprocessed': 'trusty goose-down pillow. Ra...,0
a04024b3-2fba-5ee3-ab06-dac5c10301b6,{'preprocessed': 'I can be with you until 9 p....,"{'preprocessed': '""She likes him more than you...",0
4c5e5e32-79fe-5239-ae1e-a295ca20f6c0,{'preprocessed': 'To help us most to grow If w...,"{'preprocessed': 'Onward to the next chapter, ...",0


Fit the feature sklearn transformer 
======

In [8]:
start = datetime.datetime.now()

docs = list(X_train['doc1']) + list(X_train['doc2'])
transformer = get_writeprints_transformer()
X = transformer.fit_transform(docs[:len(docs)//2]) # Usiamo 1/2 dei docs per fittare, valori più alti fanno esplodere
scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(X)

with open(TEMP_DATA_DIR + 'transformers.p', 'wb') as f:
    pickle.dump((transformer, scaler), f)

end = datetime.datetime.now()
print("Time: ", (end-start).total_seconds())

/usr/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:688: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



Time:  2192.447666


Vectorize the training data
======

In [9]:
start = datetime.datetime.now()

x1 = scaler.transform(transformer.transform(X_train['doc1']))
x2 = scaler.transform(transformer.transform(X_train['doc2']))

X_train_features = pd.DataFrame(np.abs(x1-x2).todense())

end = datetime.datetime.now()
print("Time: ", (end-start).total_seconds())

Time:  4108.768206


Vectorize the test data
=====

In [10]:
start = datetime.datetime.now()

x1 = scaler.transform(transformer.transform(X_test['doc1']))
x2 = scaler.transform(transformer.transform(X_test['doc2']))

X_test_features = pd.DataFrame(np.abs(x1-x2).todense())

end = datetime.datetime.now()
print("Time: ", (end-start).total_seconds())

Time:  1900.568006


Vectorize the Val data
====

In [11]:
start = datetime.datetime.now()
        
x1 = scaler.transform(transformer.transform(X_valid['doc1']))
x2 = scaler.transform(transformer.transform(X_valid['doc2']))
X_valid_features = pd.DataFrame(np.abs(x1-x2).todense())

end = datetime.datetime.now()
print("Time: ", (end-start).total_seconds())

Time:  1433.722064


Save to (or read from) pickle files
====

In [14]:
import pickle, pandas as pd
"""
# Per salvare
X_test_features.to_pickle(TEMP_DATA_DIR+"x_test_features.pkl")
X_valid_features.to_pickle(TEMP_DATA_DIR+"x_valid_features.pkl")
X_train_features.to_pickle(TEMP_DATA_DIR+"x_train_features.pkl")

X_test.to_pickle(TEMP_DATA_DIR+"x_test.pkl")
X_valid.to_pickle(TEMP_DATA_DIR+"x_valid.pkl")
X_train.to_pickle(TEMP_DATA_DIR+"x_train.pkl")

df.to_pickle(TEMP_DATA_DIR+"df.pkl")

"""

# Per leggere quelli già salvati
X_test_features = pd.read_pickle(TEMP_DATA_DIR+"/x_test_features.pkl")
X_valid_features = pd.read_pickle(TEMP_DATA_DIR+"/x_valid_features.pkl")
X_train_features = pd.read_pickle(TEMP_DATA_DIR+"/x_train_features.pkl")

X_test = pd.read_pickle(TEMP_DATA_DIR+"/x_test.pkl")
X_valid = pd.read_pickle(TEMP_DATA_DIR+"/x_valid.pkl")
X_train = pd.read_pickle(TEMP_DATA_DIR+"/x_train.pkl")

df = pd.read_pickle(TEMP_DATA_DIR+"/df.pkl")
y_train = X_train[['label']]
y_valid = X_valid[['label']]
y_test = X_test[['label']]

with open(TEMP_DATA_DIR + 'transformers.p', 'rb') as f:
    transformer, scaler = pickle.load(f)
    
with open(TEMP_DATA_DIR + 'ordering_metadata.p', 'rb') as f:
    train_sz, test_sz, val_sz, train_idxs = pickle.load(f)
    
with open(TEMP_DATA_DIR + 'ordering_metadata.p', 'rb') as f:
    train_idxs = pickle.load(f)


FileNotFoundError: [Errno 2] No such file or directory: 'temp_data/pan20_computed/ordering_metadata.p'

Train Classifier: Logistic Regression
===

In [50]:
y_valid

,label
id,
b6e4bcd6-d5e5-583e-a575-32ccdabae79c,0
501c2a19-9a64-5c74-9a1d-447508791eb2,0
74e4dc00-12ff-5086-b0df-b6d2997605ee,1
ce6b0f7a-2506-59be-a8b4-5aa58aa1aca8,1
ee4f1241-8bea-53c3-935e-4c6727fac04e,1
...,...
ce50255f-a919-5513-9a41-366839dc33ad,0
e05d60d5-9bb8-5f5d-b0f3-74a9ecab514e,0
4a569b1b-4ff8-54d4-9a57-0e378da68b0c,0


In [15]:
clf = LogisticRegression(random_state=42, verbose=1).fit(X_train_features, y_train)
y_pred = clf.predict(X_test_features)
y_test_new = []
for i, row in y_test.iterrows():
    y_test_new.append(row.values[0])
y_test_new = np.array(y_test_new)
#y_test = y_test_new
print(evaluate_all(y_test_new, y_pred))

/usr/lib/python3.8/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.6s finished


{'auc': 0.919, 'c@1': 0.919, 'f_05_u': 0.917, 'F1': 0.919, 'overall': 0.918}


# Train Classifier: MLP

In [27]:
clf = MLPClassifier(random_state=42, hidden_layer_sizes=100, activation='relu', solver='adam', learning_rate='constant', max_iter=20, verbose=1).fit(X_train_features, y_train.values.ravel())
y_pred = clf.predict(X_valid_features)
y_val_new = []
for i, row in y_valid.iterrows():
    y_val_new.append(row.values[0])
y_val_new = np.array(y_val_new)
#y_test = y_test_new
print(evaluate_all(y_val_new, y_pred))

Iteration 1, loss = 1.28083534
Iteration 2, loss = 0.69667992
Iteration 3, loss = 0.69574730
Iteration 4, loss = 0.69499425
Iteration 5, loss = 0.69442011
Iteration 6, loss = 0.69400266
Iteration 7, loss = 0.69370659
Iteration 8, loss = 0.69352375
Iteration 9, loss = 0.69341281
Iteration 10, loss = 0.69334811
Iteration 11, loss = 0.69330076
Iteration 12, loss = 0.69327544
Iteration 13, loss = 0.69325528
Iteration 14, loss = 0.69326019
Iteration 15, loss = 0.69324977
Iteration 16, loss = 0.69324759
Iteration 17, loss = 0.69324540
Iteration 18, loss = 0.69324772
Iteration 19, loss = 0.69324204
Iteration 20, loss = 0.69324063
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
{'auc': 0.5, 'c@1': 0.5, 'f_05_u': 0.003, 'F1': 0.001, 'overall': 0.251}


In [17]:
clf = MLPClassifier(random_state=42, hidden_layer_sizes=100, activation='logistic', solver='lbfgs', learning_rate='constant', max_iter=20, verbose=1).fit(X_train_features, y_train.values.ravel())
y_pred = clf.predict(X_valid_features)
y_val_new = []
for i, row in y_valid.iterrows():
    y_val_new.append(row.values[0])
y_val_new = np.array(y_val_new)
#y_test = y_test_new
print(evaluate_all(y_val_new, y_pred))

/usr/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



{'auc': 0.886, 'c@1': 0.886, 'f_05_u': 0.89, 'F1': 0.884, 'overall': 0.886}


In [18]:
clf = MLPClassifier(random_state=42, hidden_layer_sizes=200, activation='logistic', solver='lbfgs', learning_rate='invscaling', max_iter=50, verbose=1).fit(X_train_features, y_train.values.ravel())
y_pred = clf.predict(X_valid_features)
y_val_new = []
for i, row in y_valid.iterrows():
    y_val_new.append(row.values[0])
y_val_new = np.array(y_val_new)
#y_test = y_test_new
print(evaluate_all(y_val_new, y_pred))

{'auc': 0.907, 'c@1': 0.907, 'f_05_u': 0.905, 'F1': 0.908, 'overall': 0.907}


In [26]:
y_pred = clf.predict(X_test_features)

y_test_new = []
for i, row in y_test.iterrows():
    y_test_new.append(row.values[0])
y_test_new = np.array(y_test_new)
#y_test = y_test_new
print(evaluate_all(y_test_new, y_pred))

{'auc': 0.916, 'c@1': 0.916, 'f_05_u': 0.916, 'F1': 0.917, 'overall': 0.916}
